# Visualization of Neighborhoods Tutorial

**Objective**: 
This tutorial covers how to perform visualize neighborhoods based on two approaches.  
    
    1) Grabbing a bounding box region a vertex
    2) Grabbing n neighbors around a vertex

In [1]:
import napari
%gui qt5

In [2]:
from brainlit.utils.swc import *
from brainlit.utils.ngl_pipeline import NeuroglancerSession

In [3]:
from pathlib import Path
dest_dir = str(Path().resolve().parents[0] / "utils" / "upload")
dest_dir_segments = str(Path().resolve().parents[0] / "utils" / "upload_segments")

**Reading data from s3 path**

In [4]:
# s3_path = "s3://mouse-light-viz/precomputed_volumes/brain1_segments"
s3_path = "file://"+dest_dir_segments
seg_id,  v_id, mip = 2, 10, 1 # skeleton/neuron id, index/row of df, resolution quality
df = read_s3(s3_path, seg_id, mip)
df.head()

,sample,structure,x,y,z,r,parent
0,1,0,147.0,140.0,120.0,1.0,-1
1,4,192,148.0,139.0,120.0,1.0,1
2,7,64,148.0,139.0,120.0,1.0,4
3,8,0,148.0,139.0,120.0,1.0,7
4,14,0,148.0,139.0,121.0,1.0,8


**Converting dataframe to graph data structure to understand how vertices are connected**

In [5]:
G = df_to_graph(df)
paths = graph_to_paths(G=G)
print(f"The graph was decomposed into {len(paths)} paths")

The graph was decomposed into 179 paths


**Plotting the entire skeleton/neuron**

In [6]:
viewer = napari.Viewer(ndisplay=3)
# it is important that the number of paths put into 'data=' is at the most 1024
viewer.add_points(data=np.concatenate(paths)[804:], edge_width=2, edge_color='white', name='Skeleton 2')
viewer.add_shapes(data=paths, shape_type='path', edge_color='white', name='Skeleton 2')

<Shapes layer 'Skeleton 2 [1]' at 0x14f499208>


## Bounding Box Method

**Creating a bounding box based on a particular vertex of interest in order to get a group of neurons neighboring the vertex of interest**

In [7]:
# Create an NGL session to get the bounding box
ngl_sess = NeuroglancerSession(
    url="file://"+dest_dir, 
    url_segments="file://"+dest_dir_segments,
    mip=1
)
img, bbbox, vox = ngl_sess.pull_chunk(seg_id, v_id, 1, 1, 1)
bbox = bbbox.to_list()
box = (bbox[:3], bbox[3:])

Downloading: 100%|██████████| 3/3 [00:00<00:00, 25.82it/s]



Downloading: 100%|██████████| 4/4 [00:00<00:00, 33.75it/s]




**Getting all the coordinates of the group surrounding the vertex of interest using get_sub_neuron()**  
Note: data correction step necessary due to recentering in function!

In [8]:
G_sub = get_sub_neuron(G, box)

# preventing the re-centring of nodes to the bounding box corner (origin of the new coordinate frame)
for id in list(G_sub.nodes):
    G_sub.nodes[id]["x"] = G_sub.nodes[id]["x"] + box[0][0]
    G_sub.nodes[id]["y"] = G_sub.nodes[id]["y"] + box[0][1]
    G_sub.nodes[id]["z"] = G_sub.nodes[id]["z"] + box[0][2]
    
paths_sub = graph_to_paths(G_sub)

**Plotting vertex and vertex neighborhood**

In [17]:
# grab the coordinates of the vertex from the skeleon 
cv_skel = CloudVolume(s3_path, mip=mip)
skel = cv_skel.skeleton.get(seg_id)
vertex = skel.vertices[v_id]/cv_skel.scales[mip]["resolution"]
print(vertex)

viewer = napari.Viewer(ndisplay=3)
viewer.add_image(np.squeeze(np.array(img)))
viewer.add_points(data=np.concatenate(paths_sub), edge_width=2, edge_color='blue', name='Skeleton 2')
viewer.add_shapes(data=paths_sub, shape_type='path', edge_color='blue', name='Neighborhood',edge_width=5)

# display vertex
viewer.add_points(data=np.array(vertex), edge_width=5, edge_color='green', name='vertex')

[195.7467181  205.4302048   69.07140825]


<Points layer 'vertex' at 0x1606de908>

## Neighbors Method

In [13]:
# grab the coordinates of the vertex from the skeleon 
cv_skel = CloudVolume(s3_path, mip=mip)
skel = cv_skel.skeleton.get(seg_id)
vertex = skel.vertices[v_id]/cv_skel.scales[mip]["resolution"]
print(vertex)

# figure out where the vertex information is stored in the dataframe returned by read_s3
x, y, z = np.round((vertex))[0], np.round((vertex))[1], np.round((vertex))[2]
slice_df = (df[(df.x == x)&(df.y==y)&(df.z==z)])
v_idx = np.where((df.x == x)&(df.y==y)&(df.z==z))
v_idx = v_idx[0][0]
print(v_idx)
slice_df.head()

[195.7467181  205.4302048   69.07140825]
469


,sample,structure,x,y,z,r,parent
469,434,0,196.0,205.0,69.0,1.0,431


**On another napari window, plot again the entire neuron/skeleton.**

In [11]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_points(data=np.concatenate(paths, axis=0)[1024:], edge_width=2, edge_color='white', name='all_points')
viewer.add_shapes(data=paths, shape_type='path', edge_color='white', edge_width=3, name='skeleton')

<Shapes layer 'skeleton' at 0x150b65358>

**Get the coordinates of the neighobrs around vertex of interest using get_bfs_subgraph() and graphs_to_paths**

In [14]:
v_id_pos = v_idx  # the row index/number of the data frame
depth = 10  # the depth up to which the graph must be constructed

G_bfs=get_bfs_subgraph(G, v_id_pos, depth, df=df)  # perform Breadth first search to obtain a graph of interest
paths_bfs = graph_to_paths(G=G_bfs[0])  # obtain all the paths for visualization purposes

**Plot the vertex and vertex neighborhood**

In [15]:
x,y,z = df.iloc[v_id_pos]['x'], df.iloc[v_id_pos]['y'], df.iloc[v_id_pos]['z']

# display vertex
viewer = napari.Viewer(ndisplay=3)

viewer.add_points(data=np.array([x,y,z]), edge_width=5, edge_color='orange', name='bfs_vertex')

# display all neighbors around vertex
viewer.add_points(data=np.concatenate(paths_bfs), edge_color='red', edge_width=2, name='bfs_points')
viewer.add_shapes(data=paths_bfs, shape_type='path', edge_color='red', edge_width=3, name='bfs_sub_skeleton')

AssertionError: 

## Visualizing the output of both methods overlaid

**Create new napari window**

In [21]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_points(data=np.concatenate(paths, axis=0)[1024:], edge_width=2, edge_color='white', name='all_points')
viewer.add_shapes(data=paths, shape_type='path', edge_color='white', edge_width=3, name='full_skeleton')

<Shapes layer 'full_skeleton' at 0x186a1e2d0>

**Plot vertices and neighborhoods of each method on the same napari window to compare method outputs**

In [22]:
# display vertex of the boundary method
viewer.add_points(data=np.array(vertex), edge_width=5, edge_color='green', name='boundary_vertex')

# display all neighbors around vertex of boundary method
viewer.add_points(data=np.concatenate(paths_sub), edge_width=2, edge_color='blue', name='boundary_skeleton_pts')
viewer.add_shapes(data=paths_sub, shape_type='path', edge_color='blue', name='boundary_skeleton_lines',edge_width=5)

# display vertex of the bfs method
x,y,z = df.iloc[v_id_pos]['x'], df.iloc[v_id_pos]['y'], df.iloc[v_id_pos]['z']
viewer.add_points(data=np.array([x,y,z]), edge_width=5, edge_color='orange', name='bfs_vertex')

# display all neighbors around vertex of bfs method
viewer.add_points(data=np.concatenate(paths_bfs), edge_color='red', edge_width=2, name='bfs_skeleton_pts')
viewer.add_shapes(data=paths_bfs, shape_type='path', edge_color='red', edge_width=3, name='bfs_skeleton_lines')

<Shapes layer 'bfs_skeleton_lines' at 0x18351db50>